# Webpage English summarizer with user inputs (url, ollama-based LLM) 

In [0]:
# imports

import os
import requests
# from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama, time
from tqdm import tqdm

In [0]:
# Define system prompt.

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a detailed summary, ignoring text that might be navigation related. \
Respond in markdown, in English."

In [0]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a detailed summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [0]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [0]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [0]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = HEADERS

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [0]:
# And now: call the ollama API wrapper and return the relevant component of the response

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [0]:
# Ask the user the name of the url address

url= str(input("""
Please provide a valid url address:
https://"""))

In [0]:
# Ask the user to select a valid model

MODEL= str(input("""
Please select a LLM:
(examples: llama3.2, deepseek-r1:1.5b)
"""))

In [0]:
# Let's just make sure the model is loaded

!ollama pull {MODEL}

In [0]:
print('url: https://{0}\nModel= {1}'.format(url, MODEL))

In [0]:
summary = summarize('https://'+ url)
for summ in tqdm(summary):
    time.sleep(0.01)

In [0]:
display(Markdown(summary))